# 【使い方】

ランタイム>すべてのセルを実行（**Ctrl+F9**）によりすべてのセルを実行し、セル[2]の最後に生成された**「ファイルを選択」ボタン**から、マスク画像をアップロードしてください。ベクター変換が行われ、最後にベクター変換後のマスク画像がzipでまとめてダウンロードされます。

※ベクター変換を行う画像の枚数に応じて所要時間が異なります。

※変換に失敗する画像が含まれることがあります。変換に失敗したら、その画像だけ再度トライしてください。




# [How to use]

Run all cells by selecting Runtime > Run all cells (**Ctrl+F9**), and from the **"Select File" button** generated at the end of Cell [2], upload the mask images. The vector conversion will be performed, and at the end, the mask images after vector conversion will be compiled into a zip file for download.

Note: The required time varies depending on the number of images to be vectorized.

Note: Some images may fail to convert. If an image fails to convert, please try that image again.

In [ ]:
!pip install -q selenium

In [ ]:
import os
import zipfile
import shutil
from google.colab import files
import requests
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
uploaded_files = files.upload()
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)
temp_folder = 'temp_folder'
os.makedirs(temp_folder, exist_ok=True)

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import os
import zipfile
import requests
import shutil
import time
from selenium.common.exceptions import TimeoutException

webpage_url = "https://www.autotracer.org/"

if not os.path.exists(temp_folder):
    os.makedirs(temp_folder)

for filename in uploaded_files.keys():
    driver.get(webpage_url)

    WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.ID, 'fileupfield')))

    file_path = os.path.abspath(filename)
    file_input = driver.find_element(By.ID, 'fileupfield')
    file_input.send_keys(file_path)
    start_button = driver.find_element(By.ID, 'fbut')
    start_button.click()

    try:
        WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, "//a[contains(@href,'.svg')]")))
    except TimeoutException:
        print("The waiting for the element timed out.")
        continue

    svg_download_link = None
    try:
        svg_download_link = driver.find_element(By.XPATH, "//a[contains(@href,'.svg')]").get_attribute("href")
    except:
        pass
    if svg_download_link:
        downloaded_svg_filename = filename.split('.')[0] + '.svg'
        with open(os.path.join(temp_folder, downloaded_svg_filename), 'wb') as f:
            f.write(requests.get(svg_download_link).content)

        print(f"{downloaded_svg_filename} has been successfully downloaded.")
    else:
        print(f"The conversion of {filename} failed.")

    time.sleep(5)




In [ ]:
zip_filename = 'masksvg.zip'
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for root, dirs, files_list in os.walk(temp_folder):
        for file in files_list:
            zipf.write(os.path.join(root, file), file)

In [ ]:
# 必要なライブラリをインポート
import shutil
from google.colab import files

# 圧縮するディレクトリのパス
dir_to_zip = '/content/temp_folder'
output_filename = '/content/temp_folder.zip'

# ディレクトリを圧縮
shutil.make_archive(output_filename.replace('.zip', ''), 'zip', dir_to_zip)

# 圧縮したZIPファイルをダウンロード
files.download(output_filename)



This notebook utilizes Autotracer.org (https://www.autotracer.org/) to batch convert multiple image files into vectorized SVG files and saves them.

Copyright (c) 2024 Satoru Muro. All rights reserved.
Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:
1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
